In [1]:
import pandas as pd
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.interpolate import interp1d
from os import walk
import re
from scipy.fft import fft, fftfreq,ifft
import scipy.optimize
import scipy.signal
##Plots displayed in a separate window
%matplotlib qt
from scipy import signal

time_zero = -0.8258

Rel_Delta_X2 = 'Rel_Delta_X2'
Rel_Delta_X = 'Rel_Delta_X'
Delay = 'Delay [ps]'


def Load_data(folder,datatype = 'temperatures',temp_loc = 1):
    ####datatype = all loads the filenames to the dictionary
    ####datatype = separate2: first dictionary is polarizations (3 last letters of file) inner dictionary is temperatures (taken from temp_loc = )
    ####datatype = separate1: first dictionary is temperatures inner dictionary is polarizations (taken from temp_loc = )
        import re    ##helps finding numbers in strings
        

        """Loads data to a dictionary for datatype == 0 and to a list for datatype == 1 globals if datatype == 2"""
        
        data = {}
        path = folder +"\\"
        files = next(walk(path), (None, None, []))[2] 
        
        for f in files:
            if datatype == 'temperatures':
                temperature = re.findall(r'\d+', f)[temp_loc]          
                data.update({str(temperature):pd.read_csv(path+f,header = 9)}) 
            if datatype == 'separate':
                temperature = re.findall(r'\d+', f)[temp_loc]          
              
                polarizaton = f[-3:]
                if str(temperature) in data.keys():
                    data[str(temperature)].update({polarizaton:pd.read_csv(path+f,header = 9)})
                else:
                    data.update({str(temperature):  {polarizaton : pd.read_csv(path+f,header = 9)}  }) 
                    
            if datatype == 'separate2':
                temperature = re.findall(r'\d+', f)[temp_loc]          
             
                polarizaton = f[-3:]
                if polarizaton in data.keys():
                    data[polarizaton].update({str(temperature):pd.read_csv(path+f,header = 9)})
                else:
                    data.update({polarizaton:  {str(temperature) : pd.read_csv(path+f,header = 9)}  }) 
                    
                    
                    
            if datatype == 'separate3':
                temperature = re.findall(r'\d+', f)[temp_loc]          
                duration = f[-4:]
   
                
                if duration == 'long':
                    if 'long' in data:
                        data[duration].update({str(temperature):pd.read_csv(path+f,header = 9)})
                    else:
                        data.update({duration:  {str(temperature) : pd.read_csv(path+f,header = 9)}  })         
                else:
                    if 'short' in data:
                        data['short'].update({str(temperature):pd.read_csv(path+f,header = 9)})
                    else:
                        data.update({'short':  {str(temperature) : pd.read_csv(path+f,header = 9)}  })         
                    
            if datatype == 'all':
                print('all is all')
                data.update({f:pd.read_csv(path+f,header = 9)}) 
            #data.update({Path(f).stem.replace('-','_'):pd.read_csv(f,sheader = 9)})
            if datatype == 'LockinT':
                data.update({f:pd.read_csv(path+f,header = 6)}) 
        return(data)

def plot_PumpProbe(data,data2 = [],X = 'Delay [ps]',Y = 'Rel_Delta_X',subtract = 0,to_zero = 1,time_zero = 0,norm = 0,plot = 1,slicer = [0,-1]):
    before_t_zero = slice(-20,-1)
#     before_t_zero = slice(0,20)
    S = slice(slicer[0],slicer[1])
    dataX = data[X][S]
    dataY = data[Y][S]
    if subtract == 1:
        data2Y = data2[Y][S]
    if to_zero == 1:
        y_average = np.mean(np.array(data[Y])[before_t_zero])
        if subtract == 1:
            y_average2 = np.mean(np.array(data2[Y])[before_t_zero])
        if subtract == 2:
            y_average2 = np.mean(np.array(data2[Y])[before_t_zero])
    else:
        y_average = 0
        y_average2 = 0
    if subtract == 0:
        if norm == 0:
            if plot == 1:
                plt.plot(np.array(dataX)-time_zero,dataY-y_average)
        else:
            if plot == 1:
                plt.plot(np.array(dataX)-time_zero,(dataY-y_average)/np.max(np.abs(np.array(dataY-y_average))))

    dictionary = {X:np.array(dataX)-time_zero,Y:np.array(dataY)-y_average}   
        
    if subtract == 1:
        if plot == 1:
            
            plt.plot(np.array(dataX)-time_zero,(dataY-data2Y-y_average+y_average2)/2)
        dictionary = {X:np.array(dataX)-time_zero,Y:(dataY-y_average-data2Y+y_average2)/2}

    if subtract == 2:
        if plot == 1:
            
            plt.plot(np.array(dataX)-time_zero,(data[Y]+data2Y-y_average-y_average2)/2)
        dictionary = {X:np.array(dataX)-time_zero,Y:(dataX-y_average+data2Y-y_average2)/2}

    return(dictionary)


def plot_fft_T(data,X = 'Delay [ps]',Y = 'Rel_Delta_X',shape = 1):
    """if fit == 1 returns lorenzian fit amplitude. else returns max value"""    
    fft_dict = {}
    T = []
    Amp = []
    Freq = []
    Fwhm = []
    for arr in data:
        
        (amp,freq,fwhm) = fit_peak(data[arr],plot = 0,shape = shape)
        T.append(int(arr))
        Amp.append(amp)
        Freq.append(freq)
        Fwhm.append(fwhm)
        
        #(fft_data_x,fft_data_y) = plot_fft(data[arr])
        #fft_dict.update({arr:(fft_data_x,fft_data_y)})
        #T.append(int(arr))
        #maxfft.append(np.max(fft_data_y))
    plt.subplot(3,1,1)
    plt.plot(T,Amp,'o')
    plt.subplot(3,1,2)
    plt.plot(T,Freq,'o')
    plt.subplot(3,1,3)
    plt.plot(T,Fwhm,'o')
    return(T,Amp,Freq,Fwhm)

def plot_fft_T_max(data,X = 'Delay [ps]',Y = 'Rel_Delta_X',min_range=3.3,max_range = 3.5,plot = 1,slicer = [0,-1],background = 0):

    fft_dict = {}
    T = []
    Amp = []
    
    
   
    for arr in data:
        if background == 1:
            amp= max_peak2(fit_fft_bg(data[arr],X = X,Y = Y),X = X,Y = Y,slicer = slicer)
#         if background == 1:
        
#         ###Lazy background subtraction, removing the average between two points before and after the peak region
#             diff_1THz = np.array(data[arr][X])-1
#             ind_1THz = diff_1THz.argmin()
#             q(ind_1THz)
# #             bg = (max_peak2(data[arr],X=X,Y=Y,min_range = 2,max_range = 2.1,slicer = slicer)+max_peak2(data[arr],X=X,Y=Y,min_range = 5,max_range = 5.1,slicer = slicer))/2
#             bg = (max_peak2(data[arr],X=X,Y=Y,min_range = min_range*0.95,max_range =min_range*0.97 ,slicer = slicer)+max_peak2(data[arr],X=X,Y=Y,min_range = max_range*1.02,max_range = max_range*1.05,slicer = slicer))/2
              
        else:
            amp = max_peak2(data[arr],X=X,Y=Y,min_range = min_range,max_range = max_range,slicer = slicer)
        T.append(int(arr))
        Amp.append(amp)
        
        
        #(fft_data_x,fft_data_y) = plot_fft(data[arr])
        #fft_dict.update({arr:(fft_data_x,fft_data_y)})
        #T.append(int(arr))
        #maxfft.append(np.max(fft_data_y))
#     plt.subplot(3,1,1)
    if plot == 1:    
        plt.plot(T,Amp,'o')
#     plt.subplot(3,1,2)
#     plt.plot(T,Freq,'o')
#     plt.subplot(3,1,3)
#     plt.plot(T,Fwhm,'o')
    return(T,Amp)

def plot_fft(data,X = 'Delay [ps]',Y = 'Rel_Delta_X',returning = 0,plot = 1,slicer = [0,-1]):
    """plot: 0 => power spectrum, 1=> phase, 2=> complex number. The function returns what it plots"""
    S = slice(slicer[0],slicer[1])
    x = np.array(data[X][S])
    y = np.array(data[Y][S])

    
    # sample spacing

    T = np.abs(x[2]-x[1])
    
    # Number of sample points

    N = int(np.round(np.abs(np.max(x)-np.min(x))/T)+1)

    yf = fft(np.array(y))

    xf = fftfreq(N, T)[:N//2]
    
    if returning == 0:
        yreturn = 2.0/N * np.abs(yf[0:N//2])
        if plot == 1:
            plt.plot(xf,yreturn)

    if returning == 1:
        yreturn = np.unwrap(np.angle(yf[0:N//2]))  
        if plot == 1:
            plt.plot(xf,yreturn)

    if returning == 2:
        yreturn =  2.0/N * yf[0:N//2]
    return(np.array(xf),np.array(yreturn))



def max_peak(arr,X = 'Delay [ps]',Y = 'Rel_Delta_X',min_range = 60,max_range = 190,plot = 0,slicer = [0,-1]):
    (x,y) = plot_fft(arr,X,Y,plot = 0,slicer = slicer)
    freq_ind = np.argmax(y[min_range:max_range])
    if plot == 1:
        plt.plot(x[min_range:max_range],y[min_range:max_range],'o')
        plt.plot(x,y)
    return(np.max(y[min_range:max_range]),x[min_range:max_range][freq_ind])

def max_peak2(arr,X = 'Delay [ps]',Y = 'Rel_Delta_X',min_range = 3.3,max_range = 3.5,plot = 0,slicer = [0,-1]):
    (x,y) = plot_fft(arr,X=X,Y=Y,plot = 0,slicer = slicer)
    
    diff = np.array(x-min_range)
    f_min_ind = np.abs(diff).argmin()
    diff = np.array(x-max_range)
    f_max_ind = np.abs(diff).argmin()
    f_range = slice(f_min_ind,f_max_ind)
    
    if plot == 1:
        plt.plot(x[f_range],y[f_range],'o')
        plt.plot(x,y)
    return(np.max(y[f_range]))

def plot_LockinT(data,X = 'Temp',Y = 'Kerr',norm = 0):
    if norm ==0:
        plt.plot(data[X],data[Y])
    if norm  == 1:
        plt.plot(data[X],data[Y]/np.max(data[Y]))
        
        
def colorplot(data,Y = 'Rel_Delta_X',X = 'Delay [ps]',length = 'short'):
    T = []
    Z = []
    
    for d in data[length]:
        data = plot_PumpProbe(data[length][d],plot=0)

        x = np.array(data[length][d][X])
        T.append(int(d))
        #(xx,y) = plot_PumpProbe(data[length][d],X = X,Y = Y,plot = 0)
        to_subtract = np.mean(np.array(data[length][d][-20:-1]))
        Z.append(data[length][d][Y]) 
       
        #Z.append(y)
    T = np.array(T)
    ind = np.argsort(T)
    Z = np.array(Z)
    
    plt.pcolormesh(x,T[ind],Z[ind],shading='nearest')
    return(x,T[ind],Z[ind])

       
def colorplot2(data,Y = 'Rel_Delta_X',X = 'Delay [ps]',key = 'lin',to_zero = 1,subtract = 0,slicer = [0,-1],filter = 0,filt_phonons = 0):
    import matplotlib.colors as colors
    T = []
    Z = []
    
    if subtract == 1:
        length = 'eft'
    for d in data[key]:
        if subtract == 0:
            
            dat = plot_PumpProbe(data[key][d],plot=0,to_zero=to_zero,X=X,Y = Y,slicer = slicer)
            if filt_phonons == 1:
                dat = filter_phonons(dat,plot=0,Y = Y)
               
        if subtract == 1:
            
            dat = plot_PumpProbe(data['ght'][d],data['eft'][d],plot=0,to_zero=to_zero,X=X,Y = Y,subtract=1,slicer = slicer)

        x = np.array(dat[X])
        T.append(int(d))
        #(xx,y) = plot_PumpProbe(data[length][d],X = X,Y = Y,plot = 0)
        
        Z.append(dat[Y]) 
        #Z.append(y)
    T = np.array(T)
    ind = np.argsort(T)
    Z = np.array(Z)

    #plt.pcolormesh(x,T[ind],Z[ind],norm=colors.LogNorm(vmin=Z[ind].min(), vmax=Z[ind].max()/100),cmap='PuBu_r', shading='auto')
    map = plt.pcolormesh(x,T[ind],Z[ind], shading='auto')
#     
    return(x,T[ind],Z[ind],map)


def plot_rot_ref(data):
    T = []
    rot = []
    ref = []
    for t in data['short']:
        T.append(int(t))
        rot.append(data['short'][str(t)]['Reflectivity'][0])
        ref.append(data['short'][str(t)]['Reflectivity2'][0])
    plt.subplot(2,1,1)
    plt.plot(T,np.array(rot)/(np.array(ref)*25),'o')
    plt.subplot(2,1,2)
    plt.plot(T,np.array(ref)*25,'o')

    
def plot_left_right(data,X = 'Delay [ps]',Y = 'Rel_Delta_X',subtract = 1,plot = 1):
    to_plot = plot_PumpProbe(data['eft'],data['ght'],X = X,Y=Y,plot=plot,subtract = subtract)
    return(to_plot)

def subtract_dictionary(data,X = 'Delay [ps]',Y = 'Rel_Delta_X',subtract = 1,key1 = 'eft',key2 = 'ght',plot = 0):
    ##############create subtracted dictionary for all temperatures

    if subtract == 1:
        subdata = {}
        for t in data[key1]:
            d = plot_PumpProbe(data[key1][t],data[key2][t],subtract=1,plot = plot,Y = Y)    
            to_add = {t:d}
            subdata.update(to_add)
#     else:
#         dict = {}
#         for t in spot3_pol_run2_pol[key1]:
#             d = plot_PumpProbe(spot3_pol_run2_pol[key1][t],spot3_pol_run2_pol[key2][t],subtract=1,plot = 0)    
#             dict.update(d)
            
    return(subdata)






def filter_phonons(data,Y = 'Rel_Delta_X',X = 'Delay [ps]',f1 = 3.412,f2 = 2.67,f3 = 2.407,f4 = 4.147,Q = 2,plot =0):
     

    
    x = data[X]
    y = data[Y]

    # Create/view notch filter
    samp_freq = np.abs(1/(x[1]-x[0]))  # Sample frequency (THz)
    quality_factor = Q
    # Design a notch filter using signal.iirnotch
    b_notch1, a_notch1 = signal.iirnotch(f1, quality_factor, samp_freq)
    b_notch2, a_notch2 = signal.iirnotch(f2, quality_factor, samp_freq)
    b_notch3, a_notch3 = signal.iirnotch(f3, quality_factor, samp_freq)
    b_notch4, a_notch4 = signal.iirnotch(f4, quality_factor, samp_freq)
    
    b_lpf,a_lpf = signal.butter(3,10,fs=samp_freq)
    # Compute magnitude response of the designed filter
#     freq, h = signal.freqz(b_notch, a_notch, fs=2*np.pi)

#     fig = plt.figure(figsize=(8, 6))

    # Plot magnitude response of the filter
    # plt.plot(freq*samp_freq/(2*np.pi), 20 * np.log10(abs(h)),'r', label='Bandpass filter', linewidth='2')

    # plt.xlabel('Frequency [Hz]', fontsize=20)
    # plt.ylabel('Magnitude [dB]', fontsize=20)
    # plt.title('Notch Filter', fontsize=20)
    # plt.grid()


    # Apply notch filter to the noisy signal using signal.filtfilt
    outputSignal = signal.filtfilt(b_notch1, a_notch1, y)
    outputSignal = signal.filtfilt(b_notch2, a_notch2, outputSignal)
    outputSignal = signal.filtfilt(b_notch3, a_notch3, outputSignal)
    outputSignal = signal.filtfilt(b_notch4, a_notch4, outputSignal)
    outputSignal = signal.filtfilt(b_lpf, a_lpf, outputSignal)
    if plot == 1:
        plt.plot(x,outputSignal)
    dictionary = {X:x,Y:outputSignal}
    return dictionary

def fit_fft_bg(data,Y = 'Rel_Delta_X',X = 'Delay [ps]',f_min = 8,f_max = 12,plot = 0,slicer = [0,-1]):
    (x,y) = plot_fft(data,Y = Y,X = X,plot = plot,slicer = slicer)
    
    diff_f1THz = np.array(x)-1
    diff_f2THz = np.array(x)-2
    diff_fmin = np.array(x)-f_min
    diff_fmax = np.array(x)-f_max
    
    ind_1THz = np.abs(diff_f1THz).argmin()
    ind_2THz = np.abs(diff_f2THz).argmin()
    ind_min = np.abs(diff_fmin).argmin()
    ind_max = np.abs(diff_fmax).argmin()
    
    s = slice(ind_min,ind_max)
    s2 = slice(ind_1THz,ind_2THz)
    x_fit = np.concatenate((x[s],x[s2]))
    y_fit = np.concatenate((y[s],y[s2]))
    
    p = np.poly1d(np.polyfit(x_fit,y_fit,3))
    s3 = slice(ind_1THz,ind_max)
    if plot == 1:
        plt.subplot(211)
        plt.plot(x,y)
        plt.plot(x_fit,y_fit)
        
        plt.plot(x[s3],p(x[s3]))
        plt.subplot(212)
        plt.plot(x[s3],y[s3]-p(x[s3]))
    d = {X:x[s3],Y:y[s3]-p(x[s3])}
    return(d)



def PP_shapeexp1(x, A1,tau,c):
#     time_zero = -1.09715
    return (    (    A1*np.exp(-(x-time_zero)/tau)  +c )  *np.heaviside(x-time_zero,1))






def fit_PPexp1(arr,X = 'Delay [ps]',Y = 'Rel_Delta_X',plot = 0):
    x=arr[X]
    y=arr[Y]
#     time_zero = -1.09715
    y = y-np.mean(y[-20:-1])
    ##############  A1, tau, A2, f, phi, tau2, A3, tau3
    lower_bound = (-5e-3,0,-1)
    upper_bound = (0,6,1)
    guess = (-3e-4,3,0)
#     upper_bound = (0,20,5e-1,4,2*np.pi,20,0,20)
#     initial_guess = (-6e-4,5,1e-4,3.7,0,10,0,10)
    fitting_parameters, covariance = scipy.optimize.curve_fit(PP_shapeexp1,x ,y,bounds=(lower_bound,upper_bound),p0=guess)
    (A1,tau,c) = fitting_parameters
    if plot == 1:
        plt.subplot(2,1,1)
        plt.plot(x,y)
        plt.plot(x, (    A1*np.exp(-(x-time_zero)/tau)   +c )  *np.heaviside(x-time_zero,1))
        plt.subplot(2,1,2)
        plt.plot(x,y-(    A1*np.exp(-(x-time_zero)/tau)   +c )  *np.heaviside(x-time_zero,1))
    arr_return = {'Delay [ps]':x,'Rel_Delta_X':y-(    A1*np.exp(-(x-time_zero)/tau)      +c )  *np.heaviside(x-time_zero,1)}
    return(A1,tau,c,arr_return)


def PP_shapeexp2(x, A1,tau,A3,tau3,c):
#     time_zero = -1.09715
    return (    (    A1*np.exp(-(x-time_zero)/tau) + A3*np.exp(-(x-time_zero)/tau3)  +c )  *np.heaviside(x-time_zero,1))






def fit_PPexp2(arr,X = 'Delay [ps]',Y = 'Rel_Delta_X',plot = 0):
    x=arr[X]
    y=arr[Y]
#     time_zero = -1.09715
    y = y-np.mean(y[-20:-1])
    ##############  A1,tau,A3,tau3,c
    lower_bound = (-5e-2,0,0,0,-1)
    upper_bound = (0,100,5e-2,30000,1)
    guess = (-3e-4,2,1e-4,10,0)
#     upper_bound = (0,20,5e-1,4,2*np.pi,20,0,20)
#     initial_guess = (-6e-4,5,1e-4,3.7,0,10,0,10)
    fitting_parameters, covariance = scipy.optimize.curve_fit(PP_shapeexp2,x ,y,bounds=(lower_bound,upper_bound),p0=guess)
    (A1,tau,A3,tau3,c) = fitting_parameters
    
    y_fit =   (  (    A1*np.exp(-(x-time_zero)/tau) + A3*np.exp(-(x-time_zero)/tau3)  +c )  *np.heaviside(x-time_zero,1))
    if plot == 1:
        plt.subplot(2,1,1)
        plt.plot(x,y)

        plt.plot(x, y_fit)
        plt.subplot(2,1,2)
        plt.plot(x,y-y_fit)
    arr_return = {'Delay [ps]':x,'Rel_Delta_X':y-y_fit}
    return(A1,tau,A3,tau3,c,arr_return)




In [59]:
plot_fft()

,Delay [ps],R,X,Y,Duration,Rel_Delta_R,Rel_Delta_X,Rel_Delta_Y,Reflectivity,R2,X2,Y2,Rel_Delta_R2,Rel_Delta_X2,Rel_Delta_Y2,Reflectivity2
0,26.685128,0.000022,0.000019,-0.000012,9.456897,0.000044,0.000038,-0.000023,0.094843,0.001562,0.001555,0.000151,0.077676,0.077314,0.007492,0.020109
1,26.675743,0.000050,0.000050,-0.000009,9.712493,0.000100,0.000099,-0.000018,0.094843,0.000245,0.000244,0.000023,0.012163,0.012110,0.001130,0.020109
2,26.666359,0.000065,0.000065,-0.000010,9.958434,0.000130,0.000128,-0.000019,0.094843,0.000025,0.000024,0.000009,0.001265,0.001183,0.000448,0.020109
3,26.656974,0.000070,0.000069,-0.000010,10.202223,0.000139,0.000137,-0.000020,0.094843,0.000014,-0.000012,0.000007,0.000680,-0.000579,0.000357,0.020109
4,26.647590,0.000070,0.000069,-0.000012,10.448971,0.000140,0.000138,-0.000023,0.094843,0.000019,-0.000018,0.000007,0.000936,-0.000876,0.000330,0.020109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,-3.298103,0.000014,-0.000007,-0.000012,803.483944,0.000028,-0.000013,-0.000024,0.094843,0.000009,-0.000002,0.000009,0.000463,-0.000104,0.000451,0.020109
3196,-3.307488,0.000014,-0.000006,-0.000013,803.730014,0.000029,-0.000012,-0.000026,0.094843,0.000009,-0.000002,0.000009,0.000466,-0.000115,0.000451,0.020109
3197,-3.316872,0.000015,-0.000006,-0.000014,803.982041,0.000030,-0.000011,-0.000027,0.094843,0.000009,-0.000002,0.000009,0.000449,-0.000114,0.000434,0.020109
3198,-3.326257,0.000015,-0.000005,-0.000014,804.234036,0.000030,-0.000011,-0.000028,0.094843,0.000009,-0.000002,0.000009,0.000446,-0.000118,0.000430,0.020109


In [92]:
T = str(5)
filter_phonons
plot_PumpProbe(spot3_pol_48mW_pol['eft'][T],Y = Rel_Delta_X)
plot_PumpProbe(filter_phonons(spot3_pol_48mW_pol['eft'][T],Q = 1),Y = Rel_Delta_X)


# plot_PumpProbe(spot3_pol_48mW_pol['ght'][T])
# plot_PumpProbe(spot3_pol_25mW_pol['eft'][T],spot3_pol_25mW_pol['ght'][T],subtract=1)

{'Delay [ps]': array([26.68512762, 26.67574319, 26.66635877, ..., -3.30748768,
        -3.31687211, -3.32625653]),
 'Rel_Delta_X': array([4.73824499e-05, 5.55685773e-05, 6.17816337e-05, ...,
        4.16195919e-07, 3.84890309e-07, 2.14541270e-07])}

In [4]:
##laptop


# run1 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\Spot2',datatype='separate3')
# run2 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot2 run2',datatype='separate3',temp_loc=2)
# run3 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot 2 run3',datatype='separate3',temp_loc=2)
# cooldown = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\cooldown',datatype='LockinT')
# spot3 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\temperatures',datatype='separate3',temp_loc=2)
spot3_Fluences = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Fluences 5K',datatype='all')
spot3_19p7 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\run 2 19p7mW',datatype='separate3',temp_loc=3)
# # spot3_high_res = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\long high res 5K 19p7mW',datatype='separate3',temp_loc=2)
# # # spot3_pol = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarizations temp',datatype='separate2',temp_loc=2)
# spot3_pol_run2 = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run2',datatype='separate',temp_loc=3)
# spot3_pol_run2_pol = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run2',datatype='separate2',temp_loc=3)
spot3_pol_25mW_pol = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run25mW',datatype='separate2',temp_loc=2)
# sub19p7mW = subtract_dictionary(spot3_pol_run2_pol)
# # # subPP = subtract_dictionary(spot3_pol_run2_pol,Y = 'Rel_Delta_X2')
# sub25mW = subtract_dictionary(spot3_pol_25mW_pol)
# sub25mWPP = subtract_dictionary(spot3_pol_25mW_pol,Y = 'Rel_Delta_X2')
# # # spot3_170K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\170K',datatype='all')
# spot3_pol_48mW_pol = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run48mW',datatype='separate2',temp_loc=2)
# sub48mW = subtract_dictionary(spot3_pol_48mW_pol)
# sub48mWPP = subtract_dictionary(spot3_pol_48mW_pol,Y = 'Rel_Delta_X2')
spot3_pol_10mW_pol = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run10mW',datatype='separate2',temp_loc=2)
sub10mW = subtract_dictionary(spot3_pol_10mW_pol)
sub10mWPP = subtract_dictionary(spot3_pol_10mW_pol,Y = 'Rel_Delta_X2')


# rot_5K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\5K',temp_loc=1)
# rot_20K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\20K',temp_loc=1)
# rot_temps = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\run2 pol scan',datatype='separate2',temp_loc=0)
# time_pol_170K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\time scan 170K',temp_loc = 0)
# time_pol_5K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\time scan 5K',temp_loc = 0)

all is all
all is all
all is all
all is all
all is all
all is all


In [2]:
##Office computer
spot3_Fluences = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Fluences 5K',datatype='all')
spot3_pol_25mW_pol = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run25mW',datatype='separate2',temp_loc=2)
# sub19p7mW = subtract_dictionary(spot3_pol_run2_pol)
# # # # subPP = subtract_dictionary(spot3_pol_run2_pol,Y = 'Rel_Delta_X2')
sub25mW = subtract_dictionary(spot3_pol_25mW_pol)
sub25mWPP = subtract_dictionary(spot3_pol_25mW_pol,Y = 'Rel_Delta_X2')
# # # # spot3_170K = Load_data(r'C:\Users\Alon\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\170K',datatype='all')
spot3_pol_48mW_pol = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run48mW',datatype='separate2',temp_loc=2)
sub48mW = subtract_dictionary(spot3_pol_48mW_pol)
sub48mWPP = subtract_dictionary(spot3_pol_48mW_pol,Y = 'Rel_Delta_X2')
spot3_pol_10mW_pol = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\polarization temp run10mW',datatype='separate2',temp_loc=2)
# sub10mW = subtract_dictionary(spot3_pol_10mW_pol)
# sub10mWPP = subtract_dictionary(spot3_pol_10mW_pol,Y = 'Rel_Delta_X2')

Fluences_5K = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3 run4\Fluences 5K',temp_loc=1,datatype = 'separate2')

# rot_5K = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\5K',temp_loc=1)
# rot_20K = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\20K',temp_loc=1)
# time_pol_170K = Load_data(r'C:\Users\user\Dropbox\measurements\CrGeTe3\pump probe\CGT sample3\spot3\Polarization rotation times temps\time scan 170K',temp_loc = 0)




all is all
all is all
all is all
all is all
all is all
all is all


In [27]:
T = []
R = []
for t in spot3_pol_48mW_pol['lin']:
    R.append(spot3_pol_48mW_pol['lin'][t]['Reflectivity'][2])
    T.append(int(t))


In [30]:
plot_fft(spot3_pol_25mW_pol['lin']['5'],Y=Rel_Delta_X2)

(array([0.00000000e+00, 3.56895783e-02, 7.13791567e-02, ...,
        5.69605670e+01, 5.69962566e+01, 5.70319462e+01]),
 array([8.00269455e-03, 6.16650886e-04, 8.10928502e-04, ...,
        1.46732271e-05, 1.48651353e-05, 1.49397892e-05]))

## plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_3p8mW'])
plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_4p4mW'])
plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_7p5mW'])
plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_11p26mW'])
plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_13p8mW'])
plot_PumpProbe(spot3_Fluences['PumpProbe_Kerr_sample3_spot3_run2_Fluences_5K_19p7mW'])

In [ ]:
T = '10'
ax,fig = plt.subplots()
ax = plt.subplot(2,1,1)
plot_PumpProbe(spot3_pol_10mW_pol['lin'][T],Y = Rel_Delta_X2)
plot_PumpProbe(spot3_pol_10mW_pol['eft'][T],Y = Rel_Delta_X2)
plot_PumpProbe(spot3_pol_10mW_pol['ght'][T],Y = Rel_Delta_X2)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('DeltaR/R',fontsize = 12)
plt.ylim(-0.0065,0.001)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)

ax = plt.subplot(2,1,2)
plot_PumpProbe(spot3_pol_10mW_pol['lin'][T])
plot_PumpProbe(spot3_pol_10mW_pol['eft'][T])
plot_PumpProbe(spot3_pol_10mW_pol['ght'][T])
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('DeltaR/R',fontsize = 12)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.legend(['linear','Right','Left'])



In [5]:
pol = 'lin'
ax,fig = plt.subplots()
ax = plt.subplot(2,1,1)
Temps = ['5','15','25','45','55','75','100','150','170']
for T in Temps:
    plot_PumpProbe(spot3_pol_10mW_pol[pol][str(T)],Y = Rel_Delta_X2)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('DeltaR/R',fontsize = 12)
plt.ylim(-0.0065,0.001)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)

ax = plt.subplot(2,1,2)




for T in Temps:
    plot_PumpProbe(spot3_pol_10mW_pol[pol][str(T)],Y = Rel_Delta_X)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('Polariztion rotation [a.u]',fontsize = 12)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)
l = map(str, Temps)
plt.legend(Temps)



In [61]:
pol = 'lin'
ax,fig = plt.subplots()
ax = plt.subplot(2,1,1)
Temps = ['170','180','190','200']
for T in Temps:
    plot_PumpProbe(spot3_pol_10mW_pol[pol][str(T)],Y = Rel_Delta_X2)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('DeltaR/R',fontsize = 12)
plt.ylim(-0.0065,0.001)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)

ax = plt.subplot(2,1,2)




for T in Temps:
    plot_PumpProbe(spot3_pol_10mW_pol[pol][str(T)],Y = Rel_Delta_X)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('Polariztion rotation [a.u]',fontsize = 12)
plt.xlim(-3,25)
ax.tick_params(axis='both', which='major', labelsize=12)
l = map(str, Temps)
plt.legend(Temps)


In [146]:


# pol = 'lin'
ax,fig = plt.subplots()
# ax = plt.subplot(2,1,1)
Temps =  ['5','30','40','50','70']
# for T in Temps:
#     plot_PumpProbe(sub10mWPP[T],Y = Rel_Delta_X2)
# plt.xlabel('Delay [ps]',fontsize = 12)
# plt.ylabel('DeltaR/R',fontsize = 12)
# plt.ylim(-0.00025,0.0002)
# plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)

# ax = plt.subplot(2,1,2)




for T in Temps:
    plot_PumpProbe(sub10mW[T],Y = Rel_Delta_X)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('Polariztion rotation [a.u]',fontsize = 12)
plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)
l = map(str, Temps)
plt.legend(Temps)

In [145]:
# pol = 'lin'
ax,fig = plt.subplots()
# ax = plt.subplot(2,1,1)
Temps =  ['80','90','100','110','130']
# for T in Temps:
#     plot_PumpProbe(sub10mWPP[T],Y = Rel_Delta_X2)
# plt.xlabel('Delay [ps]',fontsize = 12)
# plt.ylabel('DeltaR/R',fontsize = 12)
# plt.ylim(-0.00025,0.0002)
# plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)

# ax = plt.subplot(2,1,2)




for T in Temps:
    plot_PumpProbe(sub10mW[T],Y = Rel_Delta_X)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('Polariztion rotation [a.u]',fontsize = 12)
plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)
l = map(str, Temps)
plt.legend(Temps)

In [108]:
# pol = 'lin'
ax,fig = plt.subplots()
# ax = plt.subplot(2,1,1)
Temps =  ['90','100','130','150','160','170']
# for T in Temps:
#     plot_PumpProbe(sub10mWPP[T],Y = Rel_Delta_X2)
# plt.xlabel('Delay [ps]',fontsize = 12)
# plt.ylabel('DeltaR/R',fontsize = 12)
# plt.ylim(-0.00025,0.0002)
# plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)

# ax = plt.subplot(2,1,2)




for T in Temps:
    plot_PumpProbe(sub10mW[T],Y = Rel_Delta_X)
plt.xlabel('Delay [ps]',fontsize = 12)
plt.ylabel('Polariztion rotation [a.u]',fontsize = 12)
plt.xlim(-3,25)
# ax.tick_params(axis='both', which='major', labelsize=12)
l = map(str, Temps)
plt.legend(Temps)

In [42]:
plt.subplot(211)
plot_PumpProbe(spot3_pol_10mW_pol['lin']['5'],Y = 'Rel_Delta_X2')
plot_PumpProbe(spot3_pol_25mW_pol['lin']['5'],Y = 'Rel_Delta_X2')
plot_PumpProbe(spot3_pol_48mW_pol['lin']['5'],Y = 'Rel_Delta_X2')
plt.subplot(212)


plot_PumpProbe(sub10mWPP['5'],Y = 'Rel_Delta_X2')
plot_PumpProbe(sub25mWPP['5'],Y = 'Rel_Delta_X2')
plot_PumpProbe(sub48mWPP['5'],Y = 'Rel_Delta_X2')

{'Delay [ps]': array([26.68512762, 26.67574319, 26.66635877, ..., -3.29810326,
        -3.30748768, -3.31687211]),
 'Rel_Delta_X2': array([-4.86313776e-02, -8.94609612e-03, -1.53001565e-03, ...,
        -1.96738402e-06,  3.17403765e-06, -2.70268203e-06])}

In [139]:
(x10mW,y10mW,z10mW,plot10mW) = colorplot2({'lin':sub10mW},slicer=[0,2700],filt_phonons=1,key = 'lin')
(x25mW,y25mW,z25mW,plot25mW) = colorplot2({'lin':sub25mW},slicer=[0,2700],filt_phonons=1,key = 'lin')
(x48mW,y48mW,z48mW,plot48mW) = colorplot2({'lin':sub48mW},slicer=[0,2700],filt_phonons=1,key = 'lin')
plt.xlabel('Delay[ps]')
plt.ylabel('Temperature [K]')

Text(0, 0.5, 'Temperature [K]')

In [144]:
plt.subplot(3,1,1)
plt.pcolormesh(x10mW,y10mW,z10mW,vmax = np.max(np.array(z10mW))*0.5)
plt.xlabel('Delay[ps]')
plt.ylabel('Temperature [K]')

plt.subplot(3,1,2)
plt.pcolormesh(x25mW,y25mW,-z25mW,vmax = np.max(np.array(-z25mW))*0.5)
plt.xlabel('Delay[ps]')
plt.ylabel('Temperature [K]')

plt.subplot(3,1,3)
plt.pcolormesh(x48mW,y48mW,-z48mW,vmax = np.max(np.array(-z48mW))*0.5)
plt.xlabel('Delay[ps]')
plt.ylabel('Temperature [K]')

<ipython-input-144-fb81b74a1116>:1: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.subplot(3,1,1)
<ipython-input-144-fb81b74a1116>:2: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x10mW,y10mW,z10mW,vmax = np.max(np.array(z10mW))*0.5)
<ipython-input-144-fb81b74a1116>:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new ins

Text(38.097222222222214, 0.5, 'Temperature [K]')

In [149]:
(x10mW,y10mW,z10mW,plot10mW) = colorplot2({'lin':sub10mW},slicer=[0,2700],filt_phonons=0,key = 'lin')
plt.pcolormesh(x10mW,y10mW,z10mW,vmax = np.max(np.array(z10mW))*0.5)
plt.xlabel('Delay[ps]')
plt.ylabel('Temperature [K]')

<ipython-input-149-4a1d77a9bf63>:2: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(x10mW,y10mW,z10mW,vmax = np.max(np.array(z10mW))*0.5)


Text(38.097222222222214, 0.5, 'Temperature [K]')

In [196]:
t = 180
# plot_PumpProbe(spot3_pol_run2_pol['lin'][str(t)])
plot_PumpProbe(spot3_pol_run2_pol['ght'][str(t)])
plot_PumpProbe(spot3_pol_run2_pol['eft'][str(t)])

{'Delay [ps]': array([53.37025523, 53.36170632, 53.3531574 , ..., -1.30860963,
        -1.31715855, -1.32570747]),
 'Rel_Delta_X': array([ 5.30272469e-05,  9.04677232e-05,  1.13731751e-04, ...,
        -6.48415532e-06, -5.28287532e-06,  3.72499332e-06])}

In [117]:
colorplot2({'lin':sub48mW},slicer=[0,2700],filt_phonons=1,key = 'lin')


(array([26.68512762, 26.67574319, 26.66635877, ...,  1.3753393 ,
         1.36595488,  1.35657045]),
 array([  5,   8,  10,  13,  15,  20,  25,  27,  30,  35,  40,  45,  50,
         53,  55,  57,  60,  63,  65,  70,  75,  80,  85,  90,  95, 100,
        105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165,
        170, 175, 180, 185, 190, 195, 200, 210, 220, 230, 240, 250, 260,
        270, 280, 290, 300]),
 array([[-1.78325704e-05, -2.17313423e-05, -2.48552959e-05, ...,
         -3.59867472e-05, -3.57451705e-05, -3.54228441e-05],
        [-1.68615051e-05, -1.96867621e-05, -2.20052616e-05, ...,
         -3.40028298e-05, -3.39477922e-05, -3.38914002e-05],
        [-1.90492479e-05, -2.18601019e-05, -2.41611298e-05, ...,
         -3.54897311e-05, -3.58006258e-05, -3.61711207e-05],
        ...,
        [-1.51827187e-05, -1.08781030e-05, -7.05480394e-06, ...,
          2.48867765e-05,  2.45042933e-05,  2.42598694e-05],
        [ 1.50861030e-05,  1.66113287e-05,  1.78600021e-05, 

In [18]:
T = []
A = []
for t in sub10mW:
    A.append(sub10mW[t]['Rel_Delta_X'][2540])
    T.append(int(t))
plt.plot(T,A,'o')

In [23]:
# plot_PumpProbe(sub10mW['5'])
# plot_PumpProbe(sub48mW['5'])
# plot_PumpProbe(sub25mW['5'])
plot_PumpProbe(spot3_pol_10mW_pol['ght']['20'])
# plot_PumpProbe(spot3_pol_25mW_pol['ght']['5'])


{'Delay [ps]': array([26.68512762, 26.67574319, 26.66635877, ..., -3.30748768,
        -3.31687211, -3.32625653]),
 'Rel_Delta_X': array([-2.66739871e-06,  1.37125586e-05,  2.44398632e-05, ...,
         8.08410260e-07, -7.88998465e-07, -1.03511379e-06])}

In [129]:
plot_fft_T_max(spot3_pol_10mW_pol['lin'],slicer=[0,2700],min_range=4,max_range=4.3)
plot_fft_T_max(spot3_pol_10mW_pol['lin'],slicer=[0,2700])
plot_fft_T_max(spot3_pol_10mW_pol['lin'],slicer=[0,2700],min_range=10,max_range=11)
# plot_fft_T_max(spot3_pol_10mW_pol['lin'],slicer=[0,2700],min_range=2,max_range=3)

([100,
  105,
  10,
  110,
  115,
  120,
  125,
  130,
  135,
  13,
  140,
  145,
  150,
  155,
  15,
  160,
  165,
  170,
  175,
  180,
  185,
  190,
  195,
  200,
  205,
  20,
  210,
  215,
  220,
  225,
  230,
  235,
  240,
  25,
  27,
  30,
  40,
  45,
  50,
  53,
  55,
  57,
  5,
  60,
  63,
  65,
  70,
  75,
  80,
  85,
  8,
  90,
  95],
 [7.538389715281434e-07,
  6.909191626014214e-07,
  1.7788373734705574e-07,
  8.262826981307121e-07,
  7.710487247600048e-07,
  7.264323717694724e-07,
  7.729529801762722e-07,
  7.695827666640687e-07,
  7.27605521240381e-07,
  2.0596567098820822e-07,
  7.584630076047307e-07,
  9.546878265930235e-07,
  1.060913238760082e-06,
  1.0794599453105831e-06,
  1.6713012379052967e-07,
  1.2145991975644354e-06,
  1.0381325840960407e-06,
  1.2893443088155107e-06,
  1.0166339207891973e-06,
  1.0576914322205677e-06,
  1.249814241435562e-06,
  8.959984811661878e-07,
  1.0730655318131378e-06,
  9.639910719718116e-07,
  8.909803477355295e-07,
  2.179286370229718e

In [354]:
plot_fft(spot3_pol_25mW_pol['lin']['105'],Y = 'Rel_Delta_X2',slicer=[0,2700])

(array([0.00000000e+00, 4.22855411e-02, 8.45710823e-02, ...,
        5.69586239e+01, 5.70009095e+01, 5.70431950e+01]),
 array([2.14857506e-03, 4.76693733e-04, 2.13255561e-04, ...,
        2.70031082e-05, 2.71411310e-05, 2.71856381e-05]))

In [421]:
# plot_PumpProbe(sub48mW['10'])
plot_PumpProbe(sub10mW[''])
# plot_PumpProbe(sub['120'])
# plot_PumpProbe(sub25mW['5'])

{'Delay [ps]': array([26.68512762, 26.67636882, 26.66761003, ..., -1.2992212 ,
        -1.30798   , -1.31673879]),
 'Rel_Delta_X': array([ 3.85852335e-06,  2.10898818e-06, -4.22690330e-07, ...,
        -8.69717176e-06, -2.55312865e-06,  1.08960830e-06])}

In [134]:
plot_fft(spot3_pol_10mW_pol['lin']['5'])
plot_fft(spot3_pol_25mW_pol['lin']['5'])

(array([0.00000000e+00, 3.56895783e-02, 7.13791567e-02, ...,
        5.69605670e+01, 5.69962566e+01, 5.70319462e+01]),
 array([3.61237808e-05, 7.20381051e-05, 3.43539161e-05, ...,
        3.32050499e-08, 2.14171645e-08, 1.47898866e-08]))

In [377]:
plot_fft(spot3_pol_25mW_pol['lin']['8'],slicer=[2800,2960])

(array([ 0.        ,  0.71356851,  1.42713701,  2.14070552,  2.85427403,
         3.56784253,  4.28141104,  4.99497955,  5.70854805,  6.42211656,
         7.13568507,  7.84925357,  8.56282208,  9.27639059,  9.9899591 ,
        10.7035276 , 11.41709611, 12.13066462, 12.84423312, 13.55780163,
        14.27137014, 14.98493864, 15.69850715, 16.41207566, 17.12564416,
        17.83921267, 18.55278118, 19.26634968, 19.97991819, 20.6934867 ,
        21.4070552 , 22.12062371, 22.83419222, 23.54776072, 24.26132923,
        24.97489774, 25.68846624, 26.40203475, 27.11560326, 27.82917177,
        28.54274027, 29.25630878, 29.96987729, 30.68344579, 31.3970143 ,
        32.11058281, 32.82415131, 33.53771982, 34.25128833, 34.96485683,
        35.67842534, 36.39199385, 37.10556235, 37.81913086, 38.53269937,
        39.24626787, 39.95983638, 40.67340489, 41.38697339, 42.1005419 ,
        42.81411041, 43.52767891, 44.24124742, 44.95481593, 45.66838444,
        46.38195294, 47.09552145, 47.80908996, 48.5

In [42]:
plot_fft(spot3_pol_48mW_pol['eft']['150'])
plot_fft(spot3_pol_25mW_pol['eft']['150'])

(array([0.00000000e+00, 3.56895783e-02, 7.13791567e-02, ...,
        5.69605670e+01, 5.69962566e+01, 5.70319462e+01]),
 array([1.59703792e-04, 1.17761378e-04, 7.13197763e-05, ...,
        4.25074620e-08, 3.52848819e-08, 3.57522491e-08]))

In [45]:
plot_fft(spot3_pol_48mW_pol['eft']['150'])
plot_fft(spot3_pol_25mW_pol['eft']['150'])

(array([0.00000000e+00, 3.56895783e-02, 7.13791567e-02, ...,
        5.69605670e+01, 5.69962566e+01, 5.70319462e+01]),
 array([1.59703792e-04, 1.17761378e-04, 7.13197763e-05, ...,
        4.25074620e-08, 3.52848819e-08, 3.57522491e-08]))

In [41]:
plot_fft(spot3_pol_48mW_pol['lin']['50'],slicer=[0,2700])
plot_fft(spot3_pol_25mW_pol['lin']['50'],slicer=[0,2700])
plot_fft(spot3_pol_10mW_pol['lin']['50'],slicer=[0,2700])


KeyError: 'lin'

In [63]:
colorplot2({'lin':sub48mW})

ValueError: not enough values to unpack (expected 2, got 1)

In [266]:
plot_fft_T_max(spot3_pol_10mW_pol['lin'],Y = 'Rel_Delta_X',slicer=[0,2700],min_range=3.35,max_range=3.45)
plot_fft_T_max(sub10mW,Y = 'Rel_Delta_X',slicer=[0,2700],min_range=3.35,max_range=3.45)

([100,
  105,
  10,
  110,
  115,
  120,
  125,
  130,
  135,
  13,
  140,
  145,
  150,
  155,
  15,
  160,
  165,
  170,
  175,
  180,
  185,
  190,
  195,
  200,
  205,
  20,
  210,
  215,
  220,
  225,
  230,
  235,
  240,
  250,
  25,
  260,
  270,
  27,
  280,
  290,
  300,
  30,
  35,
  40,
  45,
  50,
  53,
  55,
  57,
  5,
  60,
  63,
  65,
  70,
  75,
  80,
  85,
  8,
  90,
  95],
 [1.2664445201858242e-06,
  1.3178475683523954e-06,
  2.4293067157361327e-06,
  1.5546450630003513e-06,
  1.533523221717404e-06,
  1.9606459675065582e-06,
  1.5008237463518525e-06,
  1.4931335374738323e-06,
  1.4543896104735291e-06,
  2.4725969180724813e-06,
  1.4586271345206841e-06,
  1.3899202216522952e-06,
  1.0802370003389642e-06,
  1.7890925496597272e-06,
  2.5333909845160418e-06,
  1.5176713779199002e-06,
  1.5848413393701475e-06,
  1.7001233462763574e-06,
  1.6107069861845247e-06,
  1.5256650678462082e-06,
  1.2317442896914445e-06,
  7.039468388039823e-07,
  9.050691629927342e-07,
  7.5877621

In [314]:
(2900+450+220+140+400+1500+600+700)*1.17

8084.7

In [ ]:
plot_fft(spot)

In [231]:
i = 1

In [233]:
print(f'{i}')

1


In [49]:
time = 1
temp = 5
name = f'temp_is_{temp}K_time_is_{time}ps'

In [50]:
print(name)

temp_is_5K_time_is_1ps


In [52]:
t_zero = 0.045
dip = 2.39

num_loc_times = 20
times = np.linspace(t_zero,dip*2,num_loc_times)
print(times)

[0.045      0.29421053 0.54342105 0.79263158 1.04184211 1.29105263
 1.54026316 1.78947368 2.03868421 2.28789474 2.53710526 2.78631579
 3.03552632 3.28473684 3.53394737 3.78315789 4.03236842 4.28157895
 4.53078947 4.78      ]


In [53]:
a[1]

array([  5,   8,  10,  13,  15,  20,  25,  27,  30,  35,  40,  45,  50,
        53,  55,  57,  60,  63,  65,  70,  75,  80,  85,  90,  95, 100,
       105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165,
       170, 175, 180, 185, 190])

In [61]:
plt.pcolormesh(a[0],a[1],a[2],vmin = np.min(np.array(a[2]))*0.4)

In [66]:
plot_PumpProbe(spot3_pol_48mW_pol['lin'][0'])
plot_PumpProbe(spot3_pol_25mW_pol['lin']['170'])

{'Delay [ps]': array([26.68512762, 26.67636882, 26.66761003, ..., -1.30798   ,
        -1.31673879, -1.32549759]),
 'Rel_Delta_X': array([ 2.01035655e-05,  2.96740532e-05,  3.77531483e-05, ...,
         4.56924638e-06,  1.93266329e-07, -1.35817462e-06])}